In [1]:
import numpy as np
import os

with np.load(os.getcwd() + '/ train_age_image_data.npz', mmap_mode='c') as data:
    x_train = data['arr_0']
    y_train = data['arr_1']

num_classes = len(np.unique(y_train))

In [2]:
x_train.shape

(10000, 180, 180, 3)

In [3]:
import tensorflow as tf

def macro_f1(y_true, y_pred):
    # Convert predicted probabilities to class labels
    y_pred = tf.argmax(y_pred, axis=-1)
    y_true = tf.cast(y_true, tf.int32)
    y_pred = tf.cast(y_pred, tf.int32)

    # Calculate the number of true positive, false positive, and false negative predictions for each class
    true_positives = tf.cast(tf.math.count_nonzero(y_true * y_pred, axis=0), tf.float32)
    false_positives = tf.cast(tf.math.count_nonzero((1 - y_true) * y_pred, axis=0), tf.float32)
    false_negatives = tf.cast(tf.math.count_nonzero(y_true * (1 - y_pred), axis=0), tf.float32)

    # Calculate precision and recall for each class
    precision = true_positives / (true_positives + false_positives + 1e-6)
    recall = true_positives / (true_positives + false_negatives + 1e-6)

    # Calculate the F1 score for each class
    f1 = 2 * precision * recall / (precision + recall + 1e-6)

    # Calculate the macro-averaged F1 score by taking the mean of the F1 scores for each class
    macro_f1 = tf.reduce_mean(f1)
    return macro_f1

In [4]:
#from tensorflow.keras.metrics import Metric
from tensorflow.keras import layers, models
from tensorflow import keras
from tensorflow.keras.models import Sequential
from keras.callbacks import EarlyStopping
from keras.metrics import Accuracy

data_augmentation = keras.Sequential(
    [
        tf.keras.layers.Rescaling(scale = 1./255, offset=0.0),
        layers.experimental.preprocessing.RandomFlip("horizontal", input_shape=(180, 180, 3)),
        layers.experimental.preprocessing.RandomRotation(0.1),
        layers.experimental.preprocessing.RandomZoom(0.1)    
    ]
)
 
model = Sequential([
    
    data_augmentation,
    
    layers.Conv2D(32, 3, padding='valid', activation='elu', input_shape = (180,180,3)), # padding='same'
    layers.MaxPooling2D(), 
    
    layers.Conv2D(64, 3, padding='valid', activation='elu'),
    layers.MaxPooling2D(),
    
    layers.Conv2D(128, 3, padding='valid', activation='elu'),
    layers.MaxPooling2D(),
    
    layers.Conv2D(256, 3, padding='valid', activation='elu'),
    layers.MaxPooling2D(),
    
    layers.Flatten(),
    
    layers.Dense(256, activation='elu'),
    layers.Dense(256, activation='elu'),
    layers.Dropout(0.2),
    layers.Dropout(0.2),
    layers.Dense(num_classes, activation='softmax')
])

# model.compile(optimizer='nadam', loss='sparse_categorical_crossentropy', metrics=[macro_f1])
# es = EarlyStopping(monitor='macro_f1', mode = 'max', verbose = 1)

model.compile(optimizer='nadam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
es = EarlyStopping(monitor='accuracy', mode='max', verbose=1)

In [5]:
model.fit(x_train, y_train, epochs=20, callbacks=[es])
print(model.evaluate(x_train,y_train)[1])
del x_train, y_train

Epoch 1/20
313/313 [==============================] - 256s 809ms/step - loss: 1.4604 - accuracy: 0.3998
Epoch 2/20
313/313 [==============================] - 256s 817ms/step - loss: 1.2173 - accuracy: 0.4597
Epoch 3/20
313/313 [==============================] - 258s 825ms/step - loss: 1.1494 - accuracy: 0.4869
Epoch 4/20
313/313 [==============================] - 261s 834ms/step - loss: 1.1090 - accuracy: 0.5064
Epoch 5/20
313/313 [==============================] - 262s 837ms/step - loss: 1.0662 - accuracy: 0.5213
Epoch 6/20
313/313 [==============================] - 262s 837ms/step - loss: 1.0416 - accuracy: 0.5409
Epoch 7/20
313/313 [==============================] - 262s 837ms/step - loss: 1.0096 - accuracy: 0.5589
Epoch 8/20
313/313 [==============================] - 262s 837ms/step - loss: 0.9915 - accuracy: 0.5637
Epoch 9/20
313/313 [==============================] - 263s 839ms/step - loss: 0.9683 - accuracy: 0.5707
Epoch 10/20
313/313 [==============================] - 263s 840m

In [6]:
with np.load(os.getcwd() + '/ test_age_image_data.npz', mmap_mode='c') as data:
    x_test = data['arr_0']
    y_test = data['arr_1']

print(model.evaluate(x_test,y_test)[1])
del x_test,y_test

95/95 [==============================] - 17s 175ms/step - loss: 0.8513 - accuracy: 0.6569
0.6568982601165771
